In [4]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
pam_side=5  # 5 or 3 
pam_length=4  # number of bases

####################################################################################
#                                                                                  #
# Leave these values unchanged to let the script determine them automatically.         #
# Only set them if something goes wrong!                                           #
#                                                                                  #
####################################################################################
target_name = ''
target_sequence_file = '/shared/targets.yml'

In [ ]:
import yaml

def load_config_value(item_name, override_value):
    # let the user override this method with a manually-specified value
    if override_value:
        return override_value
    try:
        with open("champ.yml") as f:
            config = yaml.load(f)
            return config[item_name]
    except Exception as e:
        print(e)
        raise ValueError("We could not determine the {item_name} from champ.yml. Make sure you have a configuration file and that the value is set.".format(item_name=item_name))

In [ ]:
target_name = load_config_value('perfect_target_name', target_name)

# plot settings
fontsize = 18
tick_fontsize = 16

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import random
import itertools
from collections import defaultdict, Counter, namedtuple
from IPython.display import HTML, Image
from champ import misc, intensity, intensity_array, interactive
from Bio import SeqIO
import time
from JSAnimation import IPython_display
from matplotlib import animation
import yaml
import flabpal
from champ.adapters_cython import simple_hamming_distance
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import matplotlib.pyplot as plt
from champ.interactive import TargetSequence
from champ import plotting

In [ ]:
with open(target_sequence_file) as f:
    targets = yaml.load(f)
target = targets[target_name]
ts = TargetSequence(target, pam_side=pam_side, pam_length=pam_length)

sequence_labels = ["$%s_{%d}$" % (base, index) for base, index in ts.human_readable_indexes]
guide_sequence_labels = ["$%s_{%d}$" % (base, index) for base, index in ts.guide.human_readable_indexes]

In [ ]:
IA = intensity_array.IntensityArray()
intensity_filename = os.path.join('data', date, 'LDA_intensity_scores.txt')
IA.parse_intensities_file(intensity_filename)

target_len = len(target)
base_color = {'A': flabpal.blue, 'C': flabpal.yellow, 'G': flabpal.green, 'T': flabpal.red}
    
bases = 'ACGT'
Imin_name = 'Imin_const'
Imax_name = 'Imax_adjusted'
inten_fmt = 'LDA'
TA_calibration_strategy = 'neg_control'  # ['pam', 'neg_control', 'all_possible']
ABA_zero_strategy = 'neg_control'  # neg_control or max_concentration

dname = '_'.join([inten_fmt, Imin_name, Imax_name, TA_calibration_strategy, ABA_zero_strategy])
fig_dir = 'analysis/figs'
results_dir = 'analysis/results'

custom_results_dir = os.path.join('data', date)

In [ ]:
Kds = {}
Kd_error = {} 
ABAs = {}
ABA_error = {}
ddGs = {}
fname = 'LDA_Imin_const_Imax_adjusted_Kds_and_ABAs.txt'
fpath = os.path.join(custom_results_dir, fname)
with open(fpath) as f:
    line = next(f)
    assert line.startswith('# Target:')
    target = line.strip().split(': ')[1]
    line = next(f)
    assert line.startswith('# Neg Control')
    neg_control_target = line.strip().split(': ')[1]
    line = next(f)
    assert line.startswith('# Concentration')
    line = next(f)
    while not line.startswith('#'):
        max_concentration = float(line.strip().split()[0])
        line = next(f)
    assert line.startswith('# Seq')
    for line in f:
        if line.startswith('#'):
            continue
        words = line.strip().split()
        seq = words[0]
        assert seq not in Kds, seq
        Kd, Kd_err, ABA, ABA_err = map(float, words[1:])
        Kds[seq] = Kd
        Kd_error[seq] = Kd_err
        ABAs[seq] = max(ABA, 0.0)
        ABA_error[seq] = ABA_err
ddGs = {seq: ABAs[target] - ABA for seq, ABA in ABAs.items()}
ddG_error = ABA_error
perfect_ABA = ABAs[ts.sequence]

# Single Mismatch Affinities

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
idxs = np.arange(len(target))
width = 0.5

for i, j, mismatch_base, seq in ts.single_mismatches:
    affinity = ABAs.get(seq)
    if affinity is None:
        continue
    affinity -= perfect_ABA
    error = ABA_error[seq]
    label = mismatch_base if i == 0 else None
    bar_x_position = i - width/2.0 + width*j/4.0
    color = base_color[mismatch_base]
    error_kw = dict(ecolor='k', alpha=0.6)
    ax.bar(bar_x_position, affinity, width=width/4.0, yerr=error, color=color, error_kw=error_kw, label=label)
plotting.configure_position_penalty_axes(target, fig, ax, sequence_labels, fontsize, tick_fontsize, 'ABA', target_name)

# Double Mismatch Affinities

In [ ]:
mm = interactive.MismatchMatrix(ts.sequence)
for i, j, base_i, base_j, seq in ts.double_mismatches:
    affinity = ABAs.get(seq)
    if affinity is None:
        continue
    affinity -= perfect_ABA
    mm.set_value(i, j, base_i, base_j, affinity)
    
plotting.plot_2d_mismatches(ts.sequence, sequence_labels, mm.to_matrix())

In [ ]:
epistasis_matrix = interactive.MismatchMatrix(ts.sequence)
for downstream_mismatch_index, upstream_mismatch_index, downstream_mismatch_base, upstream_mismatch_base, sequence in ts.double_mismatches:
    double_mismatch_ABA = ABAs.get(sequence)
    upstream_sequence = ts.sequence[:upstream_mismatch_index] + upstream_mismatch_base + ts.sequence[upstream_mismatch_index+1:]
    downstream_sequence = ts.sequence[:downstream_mismatch_index] + downstream_mismatch_base + ts.sequence[downstream_mismatch_index+1:]
    upstream_single_mismatch_ABA = ABAs.get(upstream_sequence)
    downstream_single_mismatch_ABA = ABAs.get(downstream_sequence)
    if double_mismatch_ABA is not None and upstream_single_mismatch_ABA is not None and downstream_single_mismatch_ABA is not None:
        delta_double_mismatch_ABA = double_mismatch_ABA - perfect_ABA
        delta_upstream = upstream_single_mismatch_ABA - perfect_ABA
        delta_downstream = downstream_single_mismatch_ABA - perfect_ABA
        ddABA = delta_double_mismatch_ABA - delta_upstream - delta_downstream
        epistasis_matrix.set_value(downstream_mismatch_index, upstream_mismatch_index, downstream_mismatch_base, upstream_mismatch_base, ddABA)
plotting.plot_2d_mismatches(ts.sequence, sequence_labels, epistasis_matrix.to_matrix(), cmap='RdBu', force_full_bounds=True)

# Single Deletion Affinities

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
width = 0.5

for i, seq in ts.guide.single_deletions:
    sequence = ts.pam + seq if ts.pam_side == 5 else seq + ts.pam
    affinity = ABAs.get(sequence)
    if affinity is None:
        continue
    affinity -= perfect_ABA
    error = ABA_error[sequence]
    ax.bar(i, affinity, width=width, yerr=error, color=flabpal.gray, error_kw=dict(ecolor='black'))
plotting.configure_position_penalty_axes(ts.guide.sequence, fig, ax, guide_sequence_labels, fontsize, tick_fontsize, 'ABA', target_name, legend=False)

# Double Deletion Affinities

In [ ]:
from matplotlib import gridspec
import matplotlib as mpl

dm = interactive.SinglePositionMatrix(ts.guide.sequence)
for i, j, seq in ts.guide.double_deletions:
    # we add the PAM back in just to look up its affinity
    sequence = ts.pam + seq if ts.pam_side == 5 else seq + ts.pam
    affinity = ABAs.get(sequence)
    if affinity is None:
        continue
    affinity -= perfect_ABA
    dm.set_value(j, i, affinity)
        
plotting.plot_2d_deletions(ts.guide.sequence, guide_sequence_labels, dm.to_matrix())

In [ ]:
epistasis_matrix = interactive.SinglePositionMatrix(ts.guide.sequence)
for upstream_index, downstream_index, sequence in ts.guide.double_deletions:
    double_ABA = ABAs.get(ts.pam + sequence)
    upstream_sequence = ts.guide.sequence[:upstream_index] + ts.guide.sequence[upstream_index+1:]
    downstream_sequence = ts.guide.sequence[:downstream_index] + ts.guide.sequence[downstream_index+1:]
    upstream_single_ABA = ABAs.get(ts.pam + upstream_sequence)
    downstream_single_ABA = ABAs.get(ts.pam + downstream_sequence)
    if double_ABA is not None and upstream_single_ABA is not None and downstream_single_ABA is not None:
        delta_double_ABA = double_ABA - perfect_ABA
        delta_upstream = upstream_single_ABA - perfect_ABA
        delta_downstream = downstream_single_ABA - perfect_ABA
        ddABA = delta_double_ABA - (delta_upstream + delta_downstream)
        epistasis_matrix.set_value(downstream_index, upstream_index, ddABA)
plotting.plot_2d_deletions(ts.guide.sequence, guide_sequence_labels, epistasis_matrix.to_matrix(), cmap='RdBu', force_full_bounds=True)

# Single Insertion Affinities

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
idxs = np.arange(len(target))
width = 0.5

for i, j, insertion_base, seq in ts.guide.single_insertions:
    sequence = ts.pam + seq if ts.pam_side == 5 else seq + ts.pam
    affinity = ABAs.get(sequence)
    if affinity is None:
        continue
    affinity -= perfect_ABA
    error = ABA_error[sequence]
    label = insertion_base if i == 0 else None
    bar_x_position = i - width/2.0 + width*j/4.0
    color = base_color[insertion_base]
    ax.bar(bar_x_position, affinity, width=width/4.0, yerr=error, color=color, error_kw=dict(ecolor='k', alpha=0.6), label=label)
plotting.configure_position_penalty_axes(ts.guide.sequence, fig, ax, guide_sequence_labels, fontsize, tick_fontsize, 'ABA', target_name)

# Double Insertion Affinities

In [ ]:
mm = interactive.InsertionMatrix(ts.guide.sequence)
for i, j, base_i, base_j, seq in ts.guide.double_insertions:
    sequence = ts.pam + seq if ts.pam_side == 5 else seq + ts.pam
    affinity = ABAs.get(sequence)
    if affinity is None:
        continue
    affinity -= perfect_ABA
    mm.set_value(i, j, base_j, base_i, affinity)
    
plotting.plot_2d_insertions(ts.guide.sequence, guide_sequence_labels, mm.to_matrix())

In [ ]:
epistasis_matrix = interactive.InsertionMatrix(ts.guide.sequence)
for downstream_index, upstream_index, upstream_insertion_base, downstream_insertion_base, sequence in ts.guide.double_insertions:
    double_ABA = ABAs.get(ts.pam + sequence)
    upstream_sequence = ts.guide.sequence[:upstream_index] + upstream_insertion_base + ts.guide.sequence[upstream_index:]
    downstream_sequence = ts.guide.sequence[:downstream_index] + downstream_insertion_base + ts.guide.sequence[downstream_index:]
    upstream_single_ABA = ABAs.get(ts.pam + upstream_sequence)
    downstream_single_ABA = ABAs.get(ts.pam + downstream_sequence)
    if double_ABA is not None and upstream_single_ABA is not None and downstream_single_ABA is not None:
        delta_double_ABA = double_ABA - perfect_ABA
        delta_upstream = upstream_single_ABA - perfect_ABA
        delta_downstream = downstream_single_ABA - perfect_ABA
        ddABA = delta_double_ABA - (delta_upstream + delta_downstream)
        epistasis_matrix.set_value(downstream_index, upstream_index, downstream_insertion_base, upstream_insertion_base, ddABA)
plotting.plot_2d_insertions(ts.guide.sequence, guide_sequence_labels, epistasis_matrix.to_matrix(), cmap='RdBu', force_full_bounds=True)

# Complement Stretch Affinities

In [ ]:
dm = interactive.SinglePositionMatrix(ts.guide.sequence)
for start, stop, seq in ts.guide.complement_stretches:
    # we add the PAM back in just to look up its affinity
    sequence = ts.pam + seq if ts.pam_side == 5 else seq + ts.pam
    affinity = ABAs.get(sequence)
    if affinity is None:
        continue
    affinity -= perfect_ABA
    dm.set_value(stop, start, affinity)

plotting.plot_complement_stretches(ts.guide.sequence, guide_sequence_labels, dm.to_matrix())